In [60]:
import requests
import pandas as pd
import private

# User Variables
ETHERSCAN_API_KEY = private.etherscan_api
CONTRACT_ADDRESS = "0x827922686190790b37229fd06084350E74485b72"
ETHERSCAN_V2_API = 'https://api.etherscan.io/v2/api'
FILTER_ADDRESSES = [private.wal_lp.lower(), private.sickle_lp.lower()]

In [53]:
# Function to fetch NFT transfers
def get_transactions(start_block, api_key, address, action='tokentx'):
    # Just pull for given token IDs (based on Contract Address)
        params = {
            'chainid': '8453',  # base mainnet = 8453
            'module': 'account',
            'action': action,
            # 'contractaddress': contract_address,
            'address': address,
            'page': '1',
            'offset': '10000',
            'startblock': start_block,
            'endblock': 99999999,
            'sort': 'asc',
            'apikey': api_key
        }

        response = requests.get(f'{ETHERSCAN_V2_API}', params=params)
        data = response.json()
        if data["status"] != "1":
            print("Error fetching data:", data["message"], data['result'])
            return []
        
        return data["result"]

In [73]:
nft_transactions = get_transactions(25707200, private.etherscan_api, private.sickle_lp, 'tokennfttx')
token_transactions = get_transactions(25707200, private.etherscan_api, private.sickle_lp, 'tokentx')

In [69]:
# Function to process token transfers
def process_token_transfers(transfers):
    df = pd.DataFrame(transfers)
    df = df[["blockNumber", "timeStamp", "hash", "from", "to", "value", "tokenSymbol", "tokenDecimal"]]
    df["timeStamp"] = pd.to_datetime(pd.to_numeric(df["timeStamp"], errors='coerce'), unit='s')
    df["value"] = df["value"].astype(float) / (10 ** df['tokenDecimal'].astype(int))  # Convert to standard token value

    # Filter based on specified addresses
    df = df[df["from"].isin(FILTER_ADDRESSES) & df["to"].isin(FILTER_ADDRESSES)]

    return df

In [74]:
tokens = process_token_transfers(token_transactions)

In [71]:
# Function to process transactions and track series
def process_transactions(transactions, token_transfers):
    df = pd.DataFrame(transactions)
    df = df[["blockNumber", "timeStamp", "hash", "from", "to", "tokenID"]]
    df["timeStamp"] = pd.to_datetime(pd.to_numeric(df["timeStamp"], errors='coerce'), unit='s')
    
    # Identify burns (to burn address) and mints (from address is zero address)
    burn_address = "0x0000000000000000000000000000000000000000"
    df["event_type"] = df.apply(lambda row: "Mint" if row["from"] == burn_address else "Burn" if row["to"] == burn_address else "Transfer", axis=1)
    
    # Identify NFT series
    series_map = {}
    series_counter = 1
    token_df = pd.DataFrame(token_transfers)
    for _, row in token_df.iterrows():
        if row["from"].lower() == private.wal_lp.lower():
            tx_hash = row["hash"]
            matching_nft_mints = df[(df["hash"] == tx_hash) & (df["event_type"] == "Mint")]
            for _, mint in matching_nft_mints.iterrows():
                series_map[mint["tokenID"]] = series_counter
            series_counter += 1
    
    # Assign series ID to subsequent burns and mints
    df["series_id"] = df["tokenID"].map(series_map)
    for _, row in df.iterrows():
        if row["event_type"] == "Burn" and row["tokenID"] in series_map:
            tx_hash = row["hash"]
            new_mint = df[(df["hash"] == tx_hash) & (df["event_type"] == "Mint")]
            for _, mint in new_mint.iterrows():
                series_map[mint["tokenID"]] = series_map[row["tokenID"]]
    
    df["series_id"] = df["tokenID"].map(series_map)
    return df

In [75]:
nft = process_transactions(nft_transactions, tokens)